# 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np
import glob
import gc
p_list=glob.glob("data/*")
p_list

['data\\items.csv',
 'data\\item_categories.csv',
 'data\\sales_train.csv',
 'data\\sample_submission.csv',
 'data\\shops.csv',
 'data\\test.csv']

In [87]:
items = pd.read_csv(p_list[0])
item_categories = pd.read_csv(p_list[1])
sales_train =pd.read_csv(p_list[2])
submission =pd.read_csv(p_list[3])
shops =pd.read_csv(p_list[4])
test =pd.read_csv(p_list[5])

# 특성 공학

## 피쳐 한글화

In [3]:
sales_train = sales_train.rename(columns={
    "date" : "날짜",
    "date_block_num" : "월ID",
    "shop_id" : "상점ID",
    "item_id" : "상품ID",
    "item_price" : "판매가",
    "item_cnt_day" : "판매량"
})
sales_train.tail(3)

,날짜,월ID,상점ID,상품ID,판매가,판매량
2935846,14.10.2015,33,25,7459,349.0,1.0
2935847,22.10.2015,33,25,7440,299.0,1.0
2935848,03.10.2015,33,25,7460,299.0,1.0


In [4]:
shops = shops.rename(columns={
    "shop_name" : "상점명",
    "shop_id" : "상점ID"
})

In [5]:
shops.tail(3)

,상점명,상점ID
57,"Якутск Орджоникидзе, 56",57
58,"Якутск ТЦ ""Центральный""",58
59,"Ярославль ТЦ ""Альтаир""",59


In [6]:
items = items.rename(columns={
    "item_name" : "상품명",
    "item_id" : "상품ID",
    "item_category_id" : "상품분류ID"
})
items.tail(3)

,상품명,상품ID,상품분류ID
22167,Язык запросов 1С:Предприятия 8 (+CD). Хрустале...,22167,49
22168,Яйцо для Little Inu,22168,62
22169,Яйцо дракона (Игра престолов),22169,69


In [7]:
item_categories = item_categories.rename(columns=
                                        {"item_category_name": "상품분류명",
                                         "item_category_id" : "상품분류ID"
                                        })
item_categories.tail(3)

,상품분류명,상품분류ID
81,Чистые носители (шпиль),81
82,Чистые носители (штучные),82
83,Элементы питания,83


In [8]:
test = test.rename(columns={
    "shop_id" : "상점ID",
    "item_id" : "상품ID"
})
test.tail(3)

,ID,상점ID,상품ID
214197,214197,45,15757
214198,214198,45,19648
214199,214199,45,969


## 데이터 다운캐스팅

In [9]:
sales_train.memory_usage().sum() / 1024 **2

134.3926239013672

In [10]:
def downcast(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024 **2
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == "object":
            pass
        elif dtype_name == "bool":
            df[col] = df[col].astype("int8")
        elif dtype_name.startswith("int") or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col], downcast="integer")
        else:
            df[col] = pd.to_numeric(df[col], downcast="float")
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print(f"{(100*(start_mem-end_mem)/start_mem): .1f}% 압축됨")
    return df

In [11]:
all_df = [sales_train, shops, items, item_categories, test]
for df in all_df:
    df = downcast(df)

 62.5% 압축됨
 38.6% 압축됨
 54.2% 압축됨
 39.9% 압축됨
 70.8% 압축됨


In [12]:
sales_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column  Dtype  
---  ------  -----  
 0   날짜      object 
 1   월ID     int8   
 2   상점ID    int8   
 3   상품ID    int16  
 4   판매가     float32
 5   판매량     int16  
dtypes: float32(1), int16(2), int8(2), object(1)
memory usage: 50.4+ MB


## 개별데이터

### sales_train 이상치 제거

* 판매량이 음수인 데이터
* 판매가가 50000 이상인 데이터
* 판매량이 1000 이상인 데이터
* 위 데이터 들을 이상치로 간주

In [13]:
sales_train.tail(3)

,날짜,월ID,상점ID,상품ID,판매가,판매량
2935846,14.10.2015,33,25,7459,349.0,1
2935847,22.10.2015,33,25,7440,299.0,1
2935848,03.10.2015,33,25,7460,299.0,1


In [14]:
# 판매가가 0보다 큰 데이터 추출
sales_train = sales_train[sales_train["판매가"] > 0]
# 판매가가 50000보다 작은 데이터 추출
sales_train = sales_train[sales_train["판매가"] < 50000]

# 판매량이 0보다 큰 데이터 추출
sales_train = sales_train[sales_train["판매량"] > 0]
# 판매량이 50000보다 큰 데이터 추출
sales_train = sales_train[sales_train["판매량"] < 1000]

### "상점명" 피쳐의 중복데이터 수정

In [15]:
print( shops["상점명"][0] ,shops["상점명"][57] ,sep="\n")
print( shops["상점명"][1] ,shops["상점명"][58] ,sep="\n")
print( shops["상점명"][10] ,shops["상점명"][11] ,sep="\n")
print( shops["상점명"][39],shops["상점명"][40] ,sep="\n")

!Якутск Орджоникидзе, 56 фран
Якутск Орджоникидзе, 56
!Якутск ТЦ "Центральный" фран
Якутск ТЦ "Центральный"
Жуковский ул. Чкалова 39м?
Жуковский ул. Чкалова 39м²
РостовНаДону ТРК "Мегацентр Горизонт"
РостовНаДону ТРК "Мегацентр Горизонт" Островной


In [16]:
sales_train.loc[sales_train["상점ID"] == 0, "상점ID"] = 57
sales_train.loc[sales_train["상점ID"] == 1, "상점ID"] = 58
sales_train.loc[sales_train["상점ID"] == 10, "상점ID"] = 11
sales_train.loc[sales_train["상점ID"] == 39,"상점ID"] = 40

In [17]:
test.loc[test["상점ID"] == 0, "상점ID"] = 57
test.loc[test["상점ID"] == 1, "상점ID"] = 58
test.loc[test["상점ID"] == 10, "상점ID"] = 11
test.loc[test["상점ID"] == 39,"상점ID"] = 40

### shops 파생 피처 생성 및 인코딩

* LabelEncoder()

In [18]:
shops.tail(2)

,상점명,상점ID
58,"Якутск ТЦ ""Центральный""",58
59,"Ярославль ТЦ ""Альтаир""",59


In [19]:
shops["도시"] = shops["상점명"].apply(lambda x: x.split()[0])

In [20]:
shops["도시"].unique()

array(['!Якутск', 'Адыгея', 'Балашиха', 'Волжский', 'Вологда', 'Воронеж',
       'Выездная', 'Жуковский', 'Интернет-магазин', 'Казань', 'Калуга',
       'Коломна', 'Красноярск', 'Курск', 'Москва', 'Мытищи', 'Н.Новгород',
       'Новосибирск', 'Омск', 'РостовНаДону', 'СПб', 'Самара', 'Сергиев',
       'Сургут', 'Томск', 'Тюмень', 'Уфа', 'Химки', 'Цифровой', 'Чехов',
       'Якутск', 'Ярославль'], dtype=object)

In [21]:
# '!Якутск' 을 'Якутск' 으로 변경
shops.loc[shops["도시"]=='!Якутск', "도시"]="Якутск"

#### label encoding

In [22]:
# 범주형 데이터 이므로 label encoding
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
shops["도시"] = label_encoder.fit_transform(shops["도시"])

In [23]:
shops.tail(2)

,상점명,상점ID,도시
58,"Якутск ТЦ ""Центральный""",58,29
59,"Ярославль ТЦ ""Альтаир""",59,30


In [24]:
# 상점명 피처 제거
shops = shops.drop("상점명", axis=1)
shops.tail(2)

,상점ID,도시
58,58,29
59,59,30


### items 파생 피처 생성

In [25]:
items.tail(2)

,상품명,상품ID,상품분류ID
22168,Яйцо для Little Inu,22168,62
22169,Яйцо дракона (Игра престолов),22169,69


In [26]:
# 상품명 피처 제거
items = items.drop("상품명", axis=1)

In [27]:
# 상품ID 별 월ID의 min값 -> 즉 상품이 가장 처음 팔린 날

sales_train.groupby("상품ID").agg({"월ID" : "min"}).head(3)

# sales_train.groupby("상품ID").agg({"월ID" : "min"})["월ID"]
# ["월ID"]는 그룹바이 테이블의 col을 선택한 것이다.

,월ID
상품ID,
0,20
1,15
2,19


In [28]:
items["첫 판매월"] = sales_train.groupby("상품ID").agg({"월ID" : "min"})["월ID"]

items.tail(3)

,상품ID,상품분류ID,첫 판매월
22167,22167,49,8.0
22168,22168,62,0.0
22169,22169,69,14.0


In [29]:
# 한 번도 판매된 적이 없는 상품
items["첫 판매월"].isnull().sum()

368

In [30]:
# 한 번도 판매되지 않은 상품은 판매월을 미뤄서 34로 설정
print(items["첫 판매월"].max())
items["첫 판매월"] = items["첫 판매월"].fillna(34)

33.0


### item_categories 파생 피처 생성 및 인코딩
* LabelEncoder()

In [31]:
item_categories.tail(3)

,상품분류명,상품분류ID
81,Чистые носители (шпиль),81
82,Чистые носители (штучные),82
83,Элементы питания,83


In [32]:
# 상품분류명에서 대분류명 추출
item_categories["대분류"] = item_categories["상품분류명"].apply(lambda x: x.split()[0])

In [33]:
item_categories["대분류"].value_counts()

Игры          14
Книги         13
Подарки       12
Игровые        8
Аксессуары     7
Музыка         6
Программы      6
Карты          5
Кино           5
Служебные      2
Чистые         2
PC             1
Билеты         1
Доставка       1
Элементы       1
Name: 대분류, dtype: int64

In [34]:
# 대분류의 고유값 개수가 5개 미만인 것은 etc로 처리
def make_etc(x):
    if len(item_categories[item_categories["대분류"]==x]) >=5:
        return x
    else:
        return "etc"

item_categories["대분류"] = item_categories["대분류"].apply(make_etc)
item_categories.tail(3)

,상품분류명,상품분류ID,대분류
81,Чистые носители (шпиль),81,etc
82,Чистые носители (штучные),82,etc
83,Элементы питания,83,etc


#### label encoding

In [35]:
lael_encoder = LabelEncoder()

item_categories["대분류"] = label_encoder.fit_transform(item_categories["대분류"])
item_categories = item_categories.drop("상품분류명", axis=1)

In [36]:
item_categories.tail(3)

,상품분류ID,대분류
81,81,0
82,82,0
83,83,0


## 데이터 조합

### 월간 판매량, 평균 판매가

#### 월-상점-상품 데이터 생성

In [37]:
sales_train["월ID"].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33],
      dtype=int8)

In [38]:
sales_train.loc[sales_train["월ID"] == 0, "상점ID"].unique()

array([59, 25, 24, 23, 19, 22, 18, 21, 28, 27, 29, 26,  4,  6,  2,  3,  7,
       57, 58, 16, 15,  8, 11, 14, 13, 12, 53, 31, 30, 32, 35, 56, 54, 47,
       50, 42, 43, 52, 51, 41, 38, 44, 37, 46, 45], dtype=int8)

In [39]:
from itertools import product

train = []
for i in sales_train["월ID"].unique():
    all_shop = sales_train.loc[sales_train["월ID"] == i, "상점ID"].unique()
    all_item = sales_train.loc[sales_train["월ID"] == i, "상품ID"].unique()
    train.append(np.array(list(product([i], all_shop, all_item))))
col_features = ["월ID", "상점ID", "상품ID"]
train = pd.DataFrame(np.vstack(train), columns=col_features)

In [40]:
train.tail(3)

,월ID,상점ID,상품ID
10812765,33,21,7640
10812766,33,21,7632
10812767,33,21,7440


#### groupby를 통해 판매량, 판매가 생성 

In [41]:
group = sales_train.groupby(col_features).agg({"판매량": "sum", "판매가":"mean"})
group = group.reset_index()
group.tail(3)

,월ID,상점ID,상품ID,판매량,판매가
1607368,33,59,22091,1,179.0
1607369,33,59,22100,1,629.0
1607370,33,59,22102,1,1250.0


In [42]:
group = group.rename(columns={"판매량":"월간 판매량","판매가":"평균 판매가"})
train = train.merge(group, on=col_features, how="left")
train.head(3)

,월ID,상점ID,상품ID,월간 판매량,평균 판매가
0,0,59,22154,1.0,999.0
1,0,59,2552,NaN,NaN
2,0,59,2554,NaN,NaN


### 기준 피처별 상품 판매건수

#### 상품 판매건수 피처 추가

In [43]:
group = sales_train.groupby(col_features).agg({"판매량":"count"}) # 판매량을 count 하여 건수를 count
group = group.reset_index()
group.tail(3)

,월ID,상점ID,상품ID,판매량
1607368,33,59,22091,1
1607369,33,59,22100,1
1607370,33,59,22102,1


In [44]:
# 판매건수로 이름 변경
group = group.rename(columns={"판매량":"판매건수"})

train = train.merge(group, on=col_features, how="left")

train.head(3)

,월ID,상점ID,상품ID,월간 판매량,평균 판매가,판매건수
0,0,59,22154,1.0,999.0,1.0
1,0,59,2552,NaN,NaN,NaN
2,0,59,2554,NaN,NaN,NaN


## 데이터 합치기

### train_test 합치기

In [45]:
test.tail(3)

,ID,상점ID,상품ID
214197,214197,45,15757
214198,214198,45,19648
214199,214199,45,969


In [46]:
# test 데이터의  월ID를 34로 만들어두기
test["월ID"] = 34

# concat
all_data = pd.concat([train, test.drop("ID", axis=1)], ignore_index=True, keys=col_features)

# 결측값을 0 으로 대체
all_data = all_data.fillna(0)

all_data.tail(3)

,월ID,상점ID,상품ID,월간 판매량,평균 판매가,판매건수
11026965,34,45,15757,0.0,0.0,0.0
11026966,34,45,19648,0.0,0.0,0.0
11026967,34,45,969,0.0,0.0,0.0


### 데이터 병합, 다운캐스팅

In [47]:
# 데이터 병합
all_data = all_data.merge(shops, on="상점ID", how="left")
all_data = all_data.merge(items, on="상품ID", how="left")
all_data = all_data.merge(item_categories, on="상품분류ID", how="left")
all_data.tail(3)

,월ID,상점ID,상품ID,월간 판매량,평균 판매가,판매건수,도시,상품분류ID,첫 판매월,대분류
11026965,34,45,15757,0.0,0.0,0.0,20,55,0.0,7
11026966,34,45,19648,0.0,0.0,0.0,20,40,23.0,5
11026967,34,45,969,0.0,0.0,0.0,20,37,17.0,5


In [48]:
# 다운캐스팅
all_data = downcast(all_data)

 59.6% 압축됨


In [49]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11026968 entries, 0 to 11026967
Data columns (total 10 columns):
 #   Column  Dtype  
---  ------  -----  
 0   월ID     int8   
 1   상점ID    int8   
 2   상품ID    int16  
 3   월간 판매량  int16  
 4   평균 판매가  float32
 5   판매건수    int8   
 6   도시      int8   
 7   상품분류ID  int8   
 8   첫 판매월   int8   
 9   대분류     int8   
dtypes: float32(1), int16(2), int8(7)
memory usage: 241.9 MB


### 가비지 처리

In [50]:
del shops, items, item_categories
gc.collect()

0

## 시차 피처 생성
time lag feature
* 시차 피처는 보통 시계열 모델의 성능을 높여준다.
* 징검다리 피쳐 생성
    * (상품)별 평균 판매량
    * (상품+도시)별 평균 판매량
    * (상점+상품범주)별 평균 판매량
* 시차 피쳐 생성
    * 월간 판매량
    * 판매건수, 평균 판매가
    * 평균 판매량
    
* 월간 평균 판매량 - 피쳐 생성

### 징검다리 피쳐 생성

In [51]:
col_features

['월ID', '상점ID', '상품ID']

#### 월간 평균 판매량 생성 함수 정의

In [52]:
def add_mean_features(df, mean_features, col_features):
    # mean_features : 새로 만들 월간 평균 판매량 파생 피처명을 저장
    # col_features : 기준 피처
    
    # 기준 피처 확인 - 기준피처의 첫번째 요소가 "월ID"가 맞는지 확인, 기준피처의 개수가 2~3개인지 확인
    assert (col_features[0] == "월ID") and (len(col_features) in [2, 3])
    
    # 파생 피처명 설정 (feature_name)
    if len(col_features) == 2:
        feature_name = col_features[1] + "별 평균 판매량"
    else:
        feature_name = col_features[1] + " " + col_features[2] + "별 평균 판매량"
    
    # 기준 피처를 토대로 그룹화 -> 월간 평균 판매량 구하기
    group = df.groupby(col_features).agg({"월간 판매량" : "mean"})  # df[col_features] 의  월간 판매량의 평균 -> 월간 평균 판매량
    group = group.reset_index() # index reset
    group = group.rename(columns={"월간 판매량": feature_name}) # groupby col 이름을 앞서 설정해둔 feature_name으로 설정
    
    # df 와 group의 병합
    df = df.merge(group, on=col_features, how="left")
    
    # 다운캐스팅
    df = downcast(df, verbose=False) # 몇 % 압축했다는 print를 출력하지 않음
    
    # 새로 만든 feature_name 를 mean_feature 리스트에 넣음
    mean_features.append(feature_name)
    
    # 가비지 컬렉션 - 필요없는 변수 제거
    del group
    gc.collect()
    
    return df, mean_features

#### 상품별 월간평균판매량

In [53]:
item_mean_features = []

# 기준피처 : 월ID, 상품ID
all_data, item_mean_features = add_mean_features(df=all_data, mean_features=item_mean_features, col_features=["월ID","상품ID"])

# 기준피처 : 월ID, 상품ID, 도시
all_data, item_mean_features = add_mean_features(df=all_data, mean_features=item_mean_features, col_features=["월ID", "상품ID", "도시"])

In [54]:
item_mean_features

['상품ID별 평균 판매량', '상품ID 도시별 평균 판매량']

#### 상점별 월간평균판매량

In [55]:
shop_mean_features = []

# 기준피쳐 : 월ID, 상점ID, 상품분류ID
all_data, shop_mean_features = add_mean_features(df=all_data, mean_features=shop_mean_features, col_features=["월ID","상점ID","상품분류ID"])

In [56]:
shop_mean_features

['상점ID 상품분류ID별 평균 판매량']

### 시차피쳐 생성

#### 시차피쳐 생성 함수 정의
* df : 원본데이터
* lag_features_to_clip : 값의 범위를 0~20 사이로 제한할 피쳐 list
* col_features : 기준피쳐
* lag_feature : 시차를 만들 피처
* nlags : 시차
    1. 한 달 전 시차피처만 생성
    1. 한 달 전, 두 달 전 시차 피처 생성
    1. 한 달 전, 두 달 전, 세 달 전 시차 피처를 모두 생성
* clip : 새로 만든 시차 피처를 lag_features_to_clip 리스트에 저장할지 여부(T/F)

In [57]:
def add_lag_feature(df, lag_features_to_clip, col_features, lag_feature, nlags=3, clip=False):
    
    # 필요한 부분만 복사 하여 df_temp으로 생성
    df_temp = df[col_features + [lag_feature]].copy()
    
    # 시차 피처 생성
    for i in range(1, nlags+1):
        
        # 피처 명
        lag_feature_name = lag_feature + "_시차" + str(i)  # 피처명
        df_temp.columns = col_features + [lag_feature_name] # df_temp 칼럼 명 수정
        
        # 월 +1
        df_temp["월ID"] += i # 월ID 값 i 추가
        df = df.merge(df_temp.drop_duplicates(), on=col_features, how="left") # col_feature를 기준으로 병합, 같은 내용은 제거
        
        # 결측값 0
        df[lag_feature_name] = df[lag_feature_name].fillna(0)
        
        # 0~20 으로 제한할 시차 피처, 이름 저장
        if clip:
            lag_features_to_clip.append(lag_feature_name)
            
    # 다운캐스팅
    df = downcast(df, False)

    # 가비지 컬렉션
    del df_temp
    gc.collect()

    return df, lag_features_to_clip

#### 월간 판매량 시차피처 생성

In [58]:
lag_feature_to_clip = []
col_features = ["월ID", "상점ID", "상품ID"] # 기준 피처

# col_features 를 기준으로 3달치 시차 피처 생성
all_data, lag_feature_to_clip = add_lag_feature(df=all_data,
                                                                                lag_features_to_clip=lag_feature_to_clip,
                                                                                col_features=col_features,
                                                                                lag_feature="월간 판매량",
                                                                                nlags=3,
                                                                                clip=True)

In [65]:
all_data.head(2)

,월ID,상점ID,상품ID,월간 판매량,평균 판매가,판매건수,도시,상품분류ID,첫 판매월,대분류,상품ID별 평균 판매량,상품ID 도시별 평균 판매량,상점ID 상품분류ID별 평균 판매량,월간 판매량_시차1,월간 판매량_시차2,월간 판매량_시차3
0,0,59,22154,1,999.0,1,30,37,0,5,0.400000,1.0,0.088496,0,0,0
1,0,59,2552,0,0.0,0,30,58,0,7,0.022222,0.0,0.000000,0,0,0


In [66]:
# 새로만든 피처 이름
lag_feature_to_clip

['월간 판매량_시차1', '월간 판매량_시차2', '월간 판매량_시차3']

In [67]:
col_features

['월ID', '상점ID', '상품ID']

#### 판매건수, 평균판매가 , 평균판매량 시차피처 생성

In [68]:
# 판매건수 시간피처 생성
all_data, lag_feature_to_clip = add_lag_feature(df=all_data,
                                                                                    lag_features_to_clip=lag_feature_to_clip,
                                                                                    col_features=col_features,
                                                                                    lag_feature="판매건수",
                                                                                    nlags=3)
# 평균판매가 시차피처 생성
all_data, lag_feature_to_clip = add_lag_feature(df=all_data,
                                                                                    lag_features_to_clip=lag_feature_to_clip,
                                                                                    col_features=col_features,
                                                                                    lag_feature="평균 판매가",
                                                                                    nlags=3)
# 상품ID별, 상품ID+도시별   평균판매량 시차피처 생성
for item_mean_feature in item_mean_features:
    all_data, lag_feature_to_clip = add_lag_feature(df=all_data,
                                                                                    lag_features_to_clip=lag_feature_to_clip,
                                                                                    col_features=col_features,
                                                                                    lag_feature=item_mean_feature,
                                                                                    nlags=3,
                                                                                    clip=True)
# 데이터 drop
all_data = all_data.drop(item_mean_features, axis=1)

# 상점ID+상품분류ID별  평균판매량 시차피처 생성
for shop_mean_feature in shop_mean_features:
    all_data, lag_feature_to_clip = add_lag_feature(df=all_data,
                                                                                    lag_features_to_clip=lag_feature_to_clip,
                                                                                    col_features=["월ID","상점ID","상품분류ID"],
                                                                                    lag_feature=shop_mean_feature,
                                                                                    nlags=3,
                                                                                    clip=True)
# 데이터 drop
all_data = all_data.drop(shop_mean_features, axis=1)

#### 월ID 0,1,2 제거
* 3개월치 시차 피처가 생겼기 때문에 0,1,2 월에는 시차데이터가 없으므로 제거한다.
* 제거해도 괜찮은 이유는 0,1,2 월의 데이터는 이후에 남아있기 때문이다.

In [70]:
# 월ID 가 3 미만인 데이터 제거
all_data = all_data.drop(all_data[all_data["월ID"] < 3].index)

### 기타피처 추가

#### 월간 판매량 시차 피처들의 평균

In [72]:
all_data["월간 판매량 시차 평균"] = all_data[["월간 판매량_시차1","월간 판매량_시차2","월간 판매량_시차3"]].mean(axis=1)

# clip 함수는 DataFrame.clip(a,b) 으로 사용하여 a,b의 범위를 벗어나는 것은 각각 a, b로 바꾼다. 
all_data[lag_feature_to_clip + ["월간 판매량", "월간 판매량 시차 평균"]] = all_data[lag_feature_to_clip + ["월간 판매량", "월간 판매량 시차 평균"]].clip(0,20)

#### 시차 변화량

In [73]:
# 1-2 시차변화율

all_data["시차변화량1"] = all_data["월간 판매량_시차1"] / all_data["월간 판매량_시차2"]
# 0으로 나누면 np.inf 로 양의 무한대 혹은 -np.inf 음의 무한대가 생긴다. 이를 np.nan 결측값으로 바꾼뒤 fillna를 통해 0으로 채워준다.
all_data["시차변화량1"] = all_data["시차변화량1"].replace([np.inf, -np.inf], np.nan).fillna(0)

all_data["시차변화량2"] = all_data["월간 판매량_시차2"] / all_data["월간 판매량_시차3"]
# 0으로 나누면 np.inf 로 양의 무한대 혹은 -np.inf 음의 무한대가 생긴다. 이를 np.nan 결측값으로 바꾼뒤 fillna를 통해 0으로 채워준다.
all_data["시차변화량2"] = all_data["시차변화량2"].replace([np.inf, -np.inf], np.nan).fillna(0)

#### 신상여부

In [75]:
# 첫 판매월이 월ID와 같으면 신상
all_data["신상여부"] = (all_data["첫 판매월"] == all_data["월ID"])

#### 첫 판매 후 경과 기간

In [76]:
all_data["첫 판매 후 기간"] = all_data["월ID"] - all_data["첫 판매월"]

#### 월(month)

In [77]:
all_data["월"] = all_data["월ID"] % 12

## 필요없는 피처 제거

In [78]:
all_data = all_data.drop(["첫 판매월", "평균 판매가", "판매건수"], axis=1)

### 다운캐스팅

In [79]:
all_data = downcast(all_data)

 15.8% 압축됨


### 마무리

In [80]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9904582 entries, 1122386 to 11026967
Data columns (total 31 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   월ID                      int8   
 1   상점ID                     int8   
 2   상품ID                     int16  
 3   월간 판매량                   int8   
 4   도시                       int8   
 5   상품분류ID                   int8   
 6   대분류                      int8   
 7   월간 판매량_시차1               int8   
 8   월간 판매량_시차2               int8   
 9   월간 판매량_시차3               int8   
 10  판매건수_시차1                 int8   
 11  판매건수_시차2                 int8   
 12  판매건수_시차3                 int8   
 13  평균 판매가_시차1               float32
 14  평균 판매가_시차2               float32
 15  평균 판매가_시차3               float32
 16  상품ID별 평균 판매량_시차1         float32
 17  상품ID별 평균 판매량_시차2         float32
 18  상품ID별 평균 판매량_시차3         float32
 19  상품ID 도시별 평균 판매량_시차1      float32
 20  상품ID 도시별 평균 판매량_시차2      float32
 21  상

## 데이터 나누기

In [82]:
# 훈련 데이터
X_train = all_data[all_data["월ID"] < 33]
X_train = X_train.drop(["월간 판매량"], axis=1)
y_train = all_data[all_data["월ID"] < 33]["월간 판매량"]

# 검증 데이터
X_valid = all_data[all_data["월ID"] == 33]
X_valid = X_valid.drop(["월간 판매량"], axis=1)
y_valid = all_data[all_data["월ID"] == 33]["월간 판매량"]

# 테스트 데이터
X_test = all_data[all_data["월ID"] == 34]
X_test = X_test.drop(["월간 판매량"], axis=1)

del all_data
gc.collect()

40348

## 데이터 저장

In [83]:
# X_train.to_csv("data/X_train.csv")
# y_train.to_csv("data/y_train.csv")
# X_valid.to_csv("data/X_valid.csv")
# y_valid.to_csv("data/y_valid.csv")
# X_test.to_csv("data/X_test.csv")

# 베이스 라인 모델 훈련 및 성능 검증

In [86]:
%%time
import lightgbm as lgb

# 하이퍼파라미터
params = {
    "metric" : "rmse",
    "num_leaves" : 255,
    "learning_rate" : 0.005,
    "feature_fraction" : 0.75,
    "bagging_fraction" : 0.75,
    "bagging_freq" : 5,
    "force_col_wise" : True,
    "random_state" : 10
}

cat_features = ["상점ID", "도시", "상품분류ID", "대분류", "월"]

# 훈련 및 검증 데이터 셋
dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid)

# 모델 훈련
lgb_model = lgb.train(
    params=params,
    train_set=dtrain,
    num_boost_round=1500,
    valid_sets=(dtrain, dvalid),
    early_stopping_rounds=150,
    categorical_feature=cat_features,
    verbose_eval=100
)

C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['대분류', '도시', '상점ID', '상품분류ID', '월']
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and 

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Total Bins 3859
[LightGBM] [Info] Number of data points in the train set: 9452298, number of used features: 30


C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 0.297707
Training until validation scores don't improve for 150 rounds
[100]	training's rmse: 1.01223	valid_1's rmse: 0.987556
[200]	training's rmse: 0.911043	valid_1's rmse: 0.924287
[300]	training's rmse: 0.860039	valid_1's rmse: 0.899103
[400]	training's rmse: 0.831922	valid_1's rmse: 0.889376
[500]	training's rmse: 0.813469	valid_1's rmse: 0.885821
[600]	training's rmse: 0.799824	valid_1's rmse: 0.884889
[700]	training's rmse: 0.789903	valid_1's rmse: 0.884388
[800]	training's rmse: 0.781822	valid_1's rmse: 0.883989
Early stopping, best iteration is:
[746]	training's rmse: 0.786011	valid_1's rmse: 0.88382
Wall time: 5min 9s


## 예측및 결과제출

In [88]:
preds = lgb_model.predict(X_test).clip(0, 20)

submission["item_cnt_month"] = preds
submission.to_csv("data/submission.csv", index=False)